# Core: 6. Context serialization

This tutorial shows context serialization.
First of all, let's do all the necessary imports from DFF.

__Installing dependencies__

In [1]:
!python3 -m pip install -q dff[tutorials]
# Installs dff with dependencies for running tutorials
# To install the minimal version of dff, use `pip install dff`
# To install other options of dff, use `pip install dff[OPTION_NAME1,OPTION_NAME2]`
# where OPTION_NAME can be one of the options from EXTRA_DEPENDENCIES.
# e.g `pip install dff[ydb, mysql]` installs dff with dependencies for using Yandex Database and MySQL
# EXTRA_DEPENDENCIES can be found in
# https://github.com/deeppavlov/dialog_flow_framework/blob/dev/setup.py#L155

__Running tutorial__

In [2]:
import logging

from dff.script import TRANSITIONS, RESPONSE, Context, Message
import dff.script.conditions as cnd

from dff.pipeline import Pipeline
from dff.utils.testing.common import (
    check_happy_path,
    is_interactive_mode,
    run_interactive_mode,
)

This function returns the user request number.

In [3]:
def response_handler(ctx: Context, _: Pipeline, *args, **kwargs) -> Message:
    return Message(text=f"answer {len(ctx.requests)}")

In [4]:
# a dialog script
toy_script = {
    "flow_start": {
        "node_start": {
            RESPONSE: response_handler,
            TRANSITIONS: {("flow_start", "node_start"): cnd.true()},
        }
    }
}

# testing
happy_path = (
    (Message(text="hi"), Message(text="answer 1")),
    (Message(text="how are you?"), Message(text="answer 2")),
    (Message(text="ok"), Message(text="answer 3")),
    (Message(text="good"), Message(text="answer 4")),
)

Draft function that performs serialization.

In [5]:
def process_response(ctx: Context):
    ctx_json = ctx.json()
    if isinstance(ctx_json, str):
        logging.info("context serialized to json str")
    else:
        raise Exception(f"ctx={ctx_json} has to be serialized to json string")

    ctx_dict = ctx.dict()
    if isinstance(ctx_dict, dict):
        logging.info("context serialized to dict")
    else:
        raise Exception(f"ctx={ctx_dict} has to be serialized to dict")

    if not isinstance(ctx, Context):
        raise Exception(f"ctx={ctx} has to have Context type")

In [6]:
pipeline = Pipeline.from_script(
    toy_script,
    start_label=("flow_start", "node_start"),
    post_services=[process_response],
)

if __name__ == "__main__":
    check_happy_path(pipeline, happy_path)
    if is_interactive_mode():
        run_interactive_mode(pipeline)

(user) >>> text='hi'
 (bot) <<< text='answer 1'
(user) >>> text='how are you?'
 (bot) <<< text='answer 2'
(user) >>> text='ok'
 (bot) <<< text='answer 3'
(user) >>> text='good'
 (bot) <<< text='answer 4'
